In [1]:
import numpy as np
import matplotlib.pyplot as plt
from casadi import *
from copy import *

### System information: 

- nominal MPC (e.g. no noise or disturbances in process.)
- OCP: double integrator model
    - discrete-time system, linear model
    - wish to solve minimization s.t. u,x in the form:
        - $\min_{x, u} x_{N}^{\top}Q_{N}x_{N} + \sum_{k=0}^{N-1} x_{k}^{\top}Qx_{k}+u_{k}^{\top}Ru_{k}$
        - s.t.:
            - $|u_{k}| \leq 1 \quad \quad \forall_k = 0, 1,...,N-1$
            - $|y_{k}| \leq 1 \quad \quad \forall_k = 0, 1,...,N-1$
        - $x(i+1) = A x(i) + B u(i)$
        - $y(i) = C x(i)$
    - prediction horizon: $\quad 10$
    - terminal cost: $\qquad \quad x_{N}^{\top}Q_{N}x_{N}$
    - stage cost: $\qquad \qquad \sum_{k=0}^{N-1} x_{k}^{\top}Rx_{k}+u_{k}^{\top}Su_{k}$
- using off-the-shelf packages, CasADi functions to formulate the problem, Opti to solve

### Setting up the problem

Defining variables to formulate problem using helper CasADI functions

In [2]:
x_states = 2 # total number of states
x = SX.sym('x', x_states) # symbol for states

u_controls = 1 # total number of control inputs
u = SX.sym('u', u_controls) # symbol for control inputs

y_outputs = 2 # total number of outputs

pred_hori = 10 # horizon

Defining matrices

In [3]:
A, B, C = np.array([[1, 1], [0,1]]), np.array([[0.5], [1.0]]), np.array([[1, 0], [0, 1]])

Defining evolution equations

In [4]:
f = Function('f', [x, u], [(A@x + B@u)]) # dynamics, e.g. state evolution, [name, input, output]
g = Function('g', [x], [(C@x)]) # output evolution 

In [5]:
f_noise = Function('f_noise', [x, u], [((A*0.2)@x + B@u)]) # apply factor 0.2 onto A

Defining cost functions

In [6]:
Q, R, S = np.eye(x_states), np.eye(u_controls), np.eye(x_states) # weight factors
stage_cost = Function('stage_cost', [x, u], [(x.T@Q@x) + (u.T@R@u)])
terminal_cost = Function('terminal_cost', [x], [x.T@S@x])

Defining constraints and initial values of the decision variables

In [7]:
U_bound, Y_bound = 1., 1.
u_min, u_max = -U_bound * np.ones((u_controls, 1)), U_bound * np.ones((u_controls, 1))
y_min, y_max = -Y_bound * np.ones((y_outputs, 1)), Y_bound * np.ones((y_outputs, 1))

x_init, u_init, y_init = np.zeros(x_states), np.zeros(u_controls), np.zeros(y_outputs)

### OCP construction

In [8]:
opti = Opti()

#### Open-loop construction

In [9]:
# defining containers
J_ol = 0 # initialize cost/objective
X_ol, U_ol, Y_ol = [0 for _ in range(pred_hori + 1)], [0 for _ in range(pred_hori)], [0 for _ in range(pred_hori + 1)] # tracking decision variables

In [10]:
noise_J_ol = 0 # noise cost/objective
noise_X_ol, noise_U_ol, noise_Y_ol = deepcopy(X_ol), deepcopy(U_ol), deepcopy(Y_ol)

Add initial decision variables

In [11]:
initial_state = np.array([-0.75, 0.75]).reshape(-1, 1)

In [12]:
X_ol[0], Y_ol[0] = opti.parameter(x_states), opti.variable(y_outputs)
opti.set_value(X_ol[0], initial_state) # afix known initial state e.g. k = 1
opti.subject_to(Y_ol[0] == g(X_ol[0])) # afix initial output decision variable e.g. k = 1

In [13]:
noise_X_ol[0], noise_Y_ol[0] = opti.parameter(x_states), opti.variable(y_outputs)
opti.set_value(noise_X_ol[0], initial_state) # noise
opti.subject_to(noise_Y_ol[0] == g(noise_X_ol[0])) # noise

Iteratively add the rest of the decision variables

In [14]:
for k in range(pred_hori):    
    # Control input U
    U_ol[k] = opti.variable(u_controls) # control input @ step k
    opti.set_initial(U_ol[k], u_init) # initialize control input                                                            
    opti.subject_to(opti.bounded(u_min, U_ol[k], u_max)) # constraints on control input
    
    # increment stage cost
    J_ol += stage_cost(X_ol[k], U_ol[k])

    # State X
    X_ol[k + 1] = opti.variable(x_states) # state @ step k + 1
    opti.set_initial(X_ol[k + 1], x_init) # initialize state at k + 1
    
    # Output Y
    Y_ol[k + 1] = opti.variable(y_outputs) # output @ step k + 1
    opti.set_initial(Y_ol[k + 1], y_init) # initialize output at k + 1
    opti.subject_to(opti.bounded(y_min, Y_ol[k + 1], y_max)) # constraints on output 

    # constraints on dynamics and output equations
    opti.subject_to(X_ol[k + 1] == f(X_ol[k], U_ol[k]))
    opti.subject_to(Y_ol[k + 1] == g(X_ol[k + 1]))

In [15]:
# Noise decision variables
for k in range(pred_hori):    
    # Control input U
    noise_U_ol[k] = opti.variable(u_controls) # noise    
    opti.set_initial(noise_U_ol[k], u_init) # noise
    opti.subject_to(opti.bounded(u_min, noise_U_ol[k], u_max)) # noise
    
    # increment stage cost
    noise_J_ol += stage_cost(noise_X_ol[k], noise_U_ol[k]) # noise

    # State X
    noise_X_ol[k + 1] = opti.variable(x_states) # noise
    opti.set_initial(noise_X_ol[k + 1], x_init) # noise
    
    # Output Y
    noise_Y_ol[k + 1] = opti.variable(y_outputs) # output @ step k + 1
    opti.set_initial(noise_Y_ol[k + 1], y_init) # initialize output at k + 1
    opti.subject_to(opti.bounded(y_min, noise_Y_ol[k + 1], y_max)) # constraints on output 

    # constraints on dynamics and output equations
    opti.subject_to(noise_X_ol[k + 1] == f_noise(noise_X_ol[k], noise_U_ol[k])) # noise 
    opti.subject_to(noise_Y_ol[k + 1] == g(noise_X_ol[k + 1])) # noise 

Terminate cost

In [16]:
J_ol += terminal_cost(X_ol[-1]) # indexed for the last state
noise_J_ol += terminal_cost(noise_X_ol[-1]) 

Minimize the cost

In [17]:
opti.minimize(J_ol)
opti.minimize(noise_J_ol)

#### Closed-loop construction

In [44]:
sim_hori = 50 # simulation horizon

In [50]:
# setting parameters for closed loop 
X_cl[0], Y_cl[0] = opti.parameter(x_states), opti.variable(y_outputs)
noise_X_cl[0], noise_Y_cl[0] = opti.parameter(x_states), opti.variable(y_outputs)

In [51]:
# defining containers

J_cl = [0 for _ in range(sim_hori)] # store objective
X_cl, U_cl, Y_cl = [[0 for _ in range(sim_hori + 1)] for _ in range(x_states)], [[0 for _ in range(sim_hori)] for _ in range(u_controls)], [[0 for _ in range(sim_hori + 1)] for _ in range(y_outputs)]

#X_p_cl = matrix = [[[0 for _ in range(sim_hori + 1)] for _ in range(pred_hori + 1)] for _ in range(y_outputs)]

In [52]:
noise_J_cl = 0 # initialize cost/objective
noise_X_cl, noise_U_cl, noise_Y_cl = deepcopy(X_cl), deepcopy(U_cl), deepcopy(Y_cl)

#noise_X_p_cl = deepcopy(X_p_cl) # predicted states

Set the initial set

In [59]:
for i in range(x_states):
    X_cl[i][0] = initial_state[i][0]

In [60]:
for i in range(x_states):
    noise_X_cl[i][0] = initial_state[i][0]

### Solve OCP

#### [Ipopt](https://coin-or.github.io/Ipopt/): Input solver options, then solve.

In [56]:
p_opts = {'verbose': False, 'expand': True, 'print_time': 1} # options taken from K.C. --> Ipopt options to print to console
s_opts = {'max_iter': 1000, 'print_level': 1, 'tol': 1e-6}

opti.solver('ipopt', p_opts, s_opts)

soln = opti.solve()

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  18.00us (  3.00us)  16.38us (  2.73us)         6
       nlp_g  |  37.00us (  6.17us)  30.78us (  5.13us)         6
  nlp_grad_f  |  26.00us (  3.71us)  23.09us (  3.30us)         7
  nlp_hess_l  |  16.00us (  3.20us)  13.16us (  2.63us)         5
   nlp_jac_g  |  46.00us (  6.57us)  33.01us (  4.72us)         7
       total  |  10.06ms ( 10.06ms)   9.98ms (  9.98ms)         1


##### Open-loop solve

In [57]:
# Extract array --> reshape to column --> Transpose
U_ol_soln = np.array([soln.value(u) for u in U_ol]).reshape(-1, u_controls).T
X_ol_soln = np.array([soln.value(x) for x in X_ol]).reshape(-1, x_states).T
Y_ol_soln = np.array([soln.value(y) for y in Y_ol]).reshape(-1, y_outputs).T

In [61]:
noise_U_ol_soln = np.array([soln.value(u) for u in noise_U_ol]).reshape(-1, u_controls).T
noise_X_ol_soln = np.array([soln.value(x) for x in noise_X_ol]).reshape(-1, x_states).T
noise_Y_ol_soln = np.array([soln.value(y) for y in noise_Y_ol]).reshape(-1, y_outputs).T

##### Closed-loop solve

In [ ]:
for k in range(sim_hori):
    # Set parameters (just initial state)
    for i in range(x_states):
        opti.set_value(X_cl[0][i], X_cl[i][k]) # afix known initial state e.g. k = 1

    # solve OCP
    sol = opti.solve()
    
    # extract results
    U_cl_soln =[sol.value(u) for u in U_cl]
    Y_cl_soln = [sol.value(y) for y in Y_cl]
    
    # save results, only interested in the first optimal input
    for i in range(u_controls):
        U_cl[i][k] = U_cl_soln[i][0]
    for i in range(y_outputs):
        Y_cl[i][k] = Y_cl_soln[i][0]
    
    # apply input to plant
    for i in range(x_states):
        X_cl[i][k + 1] = f(X_cl[i][k], U_cl[i][k])
    

### Plotting

No noise

In [ ]:
# plot u, x on same x-axis (steps)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# X
for j in range(X_soln.shape[0]):
    ax1.plot(X_soln[j, :], '--', label=f'$x_{j}$')
if j > 0:
    ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
for i in range(U_soln.shape[0]):
    ax2.step(U_soln[i, :], '--', label=f'$u_{i}$')
if i > 0:
    ax2.legend()
ax2.set_ylabel('u, control input')
fig.suptitle('open-loop nominal OCP, no noise')
ax2.set_xlabel('k, steps')
plt.tight_layout()

Noise

In [ ]:
# plot u, x on same x-axis (steps)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# X
for j in range(noise_X_soln.shape[0]):
    ax1.plot(noise_X_soln[j, :], '--', label=f'$x_{j}$')
if j > 0:
    ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
for i in range(noise_U_soln.shape[0]):
    ax2.step(noise_U_soln[i, :], '--', label=f'$u_{i}$')
if i > 0:
    ax2.legend()
ax2.set_ylabel('u, control input')
fig.suptitle('open-loop nominal OCP, with noise')
ax2.set_xlabel('k, steps')
plt.tight_layout()